#### **Install các thư viện cần thiết**

In [ ]:
!pip install scrapy
!pip install spider3

#### **Import packages**

In [1]:
import json
import re
import pandas as pd
import scrapy

#### **Tạo project với scrapy**

In [ ]:
!scrapy startproject laptopmedia

#### **Viết class để parse url của từng trang**

In [ ]:
import scrapy
import re 

class collect_laptop_url(scrapy.Spider):
  name='laptop_urls' 
  custom_settings = {
        'CONCURRENT_REQUESTS_PER_DOMAIN': 1,
        'DOWNLOAD_DELAY': 1
        }
  handle_httpstatus_list = [403]
  
  def start_requests(self):
    urls = ['https://laptopmedia.com/specs/?q=&hPP=1000&idx=laptops&p=0']
    yield scrapy.Request(url=urls[0], callback = self.parse)
  def parse(self, response):
    for href in response.xpath('//*[@id="results_container"]/div/div/article/a[1]/@href').extract():
        yield {
          'laptop_url': href
        }
    offset = re.findall(r'p=(\d+)', response.url)[0]
    if int(offset) < 20:
      next_url = 'https://laptopmedia.com/specs/?q=&hPP=20&idx=laptops&p=' + str(int(offset)+1) 
      yield response.follow(next_url, callback = self.parse, dont_filter=True)

In [ ]:
!cd laptopmedia/laptopmedia
!scrapy crawl laptop_urls -o dataset/laptop_urls.json

-> Gặp lỗi 403 forbidden và không get được dữ liệu

In [2]:
import requests
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from lxml import html
import json
import httpx
import urllib.request
from time import sleep



-> Selenium cũng tương tự, javascript chỉ load sau khi raise exception timeout

In [36]:
import pandas as pd
def url_generator(page_path, cat_id, page):
    return f"https://tiki.vn/api/personalish/v1/blocks/listings?limit=100&is_mweb=1&aggregations=2&sort=top_seller&urlKey={page_path}&category={cat_id}&page={page}"

page = httpx.get(url_generator('sach_truyen_tieng_viet', '316', '1')).json()
for i in page['filters']:
    if i['display_name'] == 'Danh Mục Sản Phẩm':
        categories = i['values']

print(categories[1])        
       
df = []
    
all_categories = []
for cat in categories:
    cat_data = httpx.get(url_generator(cat['url_key'], cat['query_value'], '1')).json()
    df.append(pd.DataFrame(cat_data['data']))
    has_child = False
    for i in cat_data['filters']: 
        if i['display_name'] == 'Danh Mục Sản Phẩm': # Nếu chứa danh mục con thì thêm vào
            #print(i['display_name'])
            all_categories.extend(i['values'])
            has_child = True
    
    if not has_child: 
        all_categories.append(cat)
        
print(len(all_categories))


{'count': 14195, 'display_value': 'Sách kinh tế', 'query_value': 846, 'url_key': 'sach-kinh-te'}
100


In [31]:
categories[0]

{'count': 27211,
 'display_value': 'Sách văn học',
 'query_value': 839,
 'url_key': 'sach-van-hoc'}

In [61]:
all_categories[0]

{'count': 198,
 'display_value': 'Du ký',
 'query_value': 4933,
 'url_key': 'du-ky'}

api limit = 100 -> Mỗi lần request lấy được 100 giá trị</br>
Cần lấy giá trị tối đa của số sách có thể có trong một thể loại 

**Lấy id của các cuốn sách ở từng thể loại**

In [73]:
# all_categories = pd.DataFrame(all_categories)
# display(all_categories)
def get_books_id(cat):
    n = (cat['count']//100) + 2
    books = {
        'Thể loại': cat['display_value'],
        'Book_id': []
    }
    for i in range(1, n):
        response = httpx.get(url_generator(cat['url_key'], cat['query_value'], i)).json()
        if 'data' in response:
            for book in response['data']:
                books['Book_id'].append({'id': book['id']})
    
    return books


In [75]:
books_id_all_cate = []
for cate in all_categories:
    print(cate['display_value'])
    books_id_all_cate.append(get_books_id(cate))

Du ký
Light novel
Phê Bình - Lý Luận Văn Học
Phóng sự - Ký sự - Bút ký
Tác phẩm kinh điển
Thơ
Tiểu sử - Hồi ký
Tiểu Thuyết
Tranh Truyện
Truyện cổ tích - Ngụ ngôn
Truyện cười
Truyện dài
Truyện đam mỹ
Truyện Giả tưởng - Huyền Bí - Phiêu Lưu
Truyện kiếm hiệp
Truyện kinh dị
Truyện ngắn - Tản văn - Tạp Văn
Truyện ngôn tình
Truyện trinh thám
Ca dao - Tục ngữ
Sách ảnh
Bài học kinh doanh
Sách doanh nhân
Sách khởi nghiệp
Sách kinh tế học
Sách kỹ năng làm việc
Sách Marketing - Bán hàng 
Sách quản trị, lãnh đạo
Sách quản trị nhân lực
Sách tài chính, kế toán
Sách tài chính, tiền tệ
Đạo đức - Kỹ năng sống
Kiến thức - Bách khoa
Tô màu - Luyện chữ  
Truyện cổ tích
Truyện kể cho bé
Truyện tranh Ehon
Truyện tranh thiếu nhi
Văn học thiếu nhi
Sách tư duy - Kỹ năng sống
Sách nghệ thuật sống đẹp
Sách hướng nghiệp - Kỹ năng mềm
Sách Thai Giáo
Sách Kiến Thức - Kỹ Năng Cho Trẻ
Sách Làm Cha Mẹ
Sách Dinh Dưỡng - Sức Khỏe Cho Bé
Sách Giáo Khoa Cấp 1
Sách Giáo Khoa Cấp 2
Sách Giáo Khoa Cấp 3
Giáo Trình Đại Học - 

In [146]:
books_id = pd.DataFrame(books_id_all_cate)
books_id.to_csv('books_id.csv', encoding='utf-8-sig')
with open('books_id.json', 'w', encoding='utf-8-sig') as file:
    books_id.to_json('books_id.json', force_ascii=False)
    
books_id.head()
leng = 0
for cate in books_id['Book_id']:
    for book in cate:
        # print(book)
        leng+=1
print(leng)

95926


#### **Lấy thông tin của từng cuốn sách dựa vào id đã thu thập**

In [149]:
book_df = []

for cate in books_id_all_cate[:20]:
    for book in cate['Book_id']:
        url = f'https://tiki.vn/api/v2/products/{book["id"]}'
        try:
            response = httpx.get(url)
            raw_book = response.json()
            print(response.status_code)
            if response.status_code == 200:
                book_info = {
                    'id': raw_book['id'],
                    'master_id': raw_book['master_id'],
                    'sku': raw_book['sku'],
                    'name': raw_book['name'],
                    'short_url': raw_book['short_url'],
                    'short_description': raw_book['short_description'],
                    'book_cover': raw_book['book_cover'],
                    'price': raw_book['price'],
                    'original_price': raw_book['original_price'],
                    'discount_rate': raw_book['discount_rate'],
                    'rating_average': raw_book['rating_average'],
                    'review_count': raw_book['review_count'],
                    'inventory_type': raw_book['inventory_type'],
                    'productset_group_name': raw_book['productset_group_name'],
                    'day_ago_created': raw_book['day_ago_created'],
                    'categories': raw_book['categories']['name'],
                    'all_time_quantity_sold': raw_book['all_time_quantity_sold'] if 'all_time_quantity_sold' in book else 0
                    
                }
                name_authors = []
                if 'authors' in raw_book:
                    for author in raw_book['authors']:
                        name_authors.append(author['name'])
                book_info['authors'] = ', '.join(name_authors) 
                
                if 'specifications' in book:
                    for i in book['specifications'][0]['attributes']:
                        book_info[i['code']] = i['code']
                        book_info[i['value']] = i['value']
                book_df.append(book_info)              
        except:
            book_df.append(raw_book)
            print('ERROR:', book)

200
200
200
ERROR: {'id': 205279963}
200
200
ERROR: {'id': 204649516}
200
200
200
200
200
200
200
200
ERROR: {'id': 205288789}
200
200
200
200
ERROR: {'id': 205293454}
200
200
200
200
ERROR: {'id': 177315574}
200
200
200
200
200
200
200
200
200
200
200
ERROR: {'id': 62349850}
ERROR: {'id': 1058052}
200
200
200
200
200
200
ERROR: {'id': 171580723}
ERROR: {'id': 197897214}
ERROR: {'id': 4708867}
ERROR: {'id': 127170159}
200
ERROR: {'id': 8011013}
200
ERROR: {'id': 51550236}
200
ERROR: {'id': 50214757}
200
ERROR: {'id': 204416968}
200
200
ERROR: {'id': 174283444}
200
200
200
200
200
ERROR: {'id': 15592528}
ERROR: {'id': 48753010}
200
ERROR: {'id': 23870341}
ERROR: {'id': 181991026}
200
ERROR: {'id': 14523892}
ERROR: {'id': 195715683}
ERROR: {'id': 167462132}
ERROR: {'id': 174182720}
ERROR: {'id': 507422}
ERROR: {'id': 565422}
ERROR: {'id': 137941512}
ERROR: {'id': 193900116}
ERROR: {'id': 127692271}
200
200
200
200
200
ERROR: {'id': 186277183}
200
ERROR: {'id': 115254287}
200
200
200
200


<font color=red>Mất khoảng 185 phút cho 1/5 dữ liệu 😅</font>

In [315]:
count = 0
for i in books_id_all_cate[:20]:
    count+= len(i['Book_id'])
print(count)

18316


In [328]:
book_df = pd.DataFrame(book_df)
book_df.to_csv('books_data1.csv', encoding='utf-8-sig')
print(book_df.shape)
display(book_df.head())

(18316, 18)


,id,master_id,sku,name,short_url,short_description,book_cover,price,original_price,discount_rate,rating_average,review_count,inventory_type,productset_group_name,day_ago_created,categories,all_time_quantity_sold,authors
0,204317934,204317934,9972713357587,1111 - Nhật Ký Sáu Vạn Dặm Trên Yên Xe Cà Tàng,https://tiki.vn/product-p204317934.html?spid=2...,Trần Đặng Đăng Khoa bắt đầu hành trình vạn dặm...,None,262000,325000,19,5.0,54,instock,Nhà Sách Tiki/Sách tiếng Việt/Sách văn học/Du ký,27,Du ký,0,Trần Đặng Đăng Khoa
1,193209826,193209826,5001141446127,Con Đường Tơ Lụa - Vạn Dặm Xa Từ Pakistan Tới ...,https://tiki.vn/product-p193209826.html?spid=1...,Con đường tơ lụa là một trong những tuyến đườn...,None,83000,129000,36,4.8,5,instock,Nhà Sách Tiki/Sách tiếng Việt/Sách văn học/Du ký,121,Du ký,0,Trần Hồng Ngọc
2,193209826,193209826,5001141446127,Con Đường Tơ Lụa - Vạn Dặm Xa Từ Pakistan Tới ...,https://tiki.vn/product-p193209826.html?spid=1...,Con đường tơ lụa là một trong những tuyến đườn...,None,83000,129000,36,4.8,5,instock,Nhà Sách Tiki/Sách tiếng Việt/Sách văn học/Du ký,121,Du ký,0,Trần Hồng Ngọc
3,3577373,3577373,3242205815295,Ta Ba Lô Trên Đất Á,https://tiki.vn/product-p3577373.html?spid=204...,Ta Ba Lô Trên Đất ÁQuyển sách đầu tiên của Ros...,None,102600,108000,5,5.0,16,backorder,Nhà Sách Tiki/Sách tiếng Việt/Sách văn học/Du ký,1581,Du ký,0,
4,3577373,3577373,3242205815295,Ta Ba Lô Trên Đất Á,https://tiki.vn/product-p3577373.html?spid=204...,Ta Ba Lô Trên Đất ÁQuyển sách đầu tiên của Ros...,None,102600,108000,5,5.0,16,backorder,Nhà Sách Tiki/Sách tiếng Việt/Sách văn học/Du ký,1581,Du ký,0,


In [330]:
book_df.duplicated().sum()

10372

In [334]:
book_df.drop_duplicates()

,id,master_id,sku,name,short_url,short_description,book_cover,price,original_price,discount_rate,rating_average,review_count,inventory_type,productset_group_name,day_ago_created,categories,all_time_quantity_sold,authors
0,204317934,204317934,9972713357587,1111 - Nhật Ký Sáu Vạn Dặm Trên Yên Xe Cà Tàng,https://tiki.vn/product-p204317934.html?spid=2...,Trần Đặng Đăng Khoa bắt đầu hành trình vạn dặm...,None,262000,325000,19,5.0,54,instock,Nhà Sách Tiki/Sách tiếng Việt/Sách văn học/Du ký,27,Du ký,0,Trần Đặng Đăng Khoa
1,193209826,193209826,5001141446127,Con Đường Tơ Lụa - Vạn Dặm Xa Từ Pakistan Tới ...,https://tiki.vn/product-p193209826.html?spid=1...,Con đường tơ lụa là một trong những tuyến đườn...,None,83000,129000,36,4.8,5,instock,Nhà Sách Tiki/Sách tiếng Việt/Sách văn học/Du ký,121,Du ký,0,Trần Hồng Ngọc
3,3577373,3577373,3242205815295,Ta Ba Lô Trên Đất Á,https://tiki.vn/product-p3577373.html?spid=204...,Ta Ba Lô Trên Đất ÁQuyển sách đầu tiên của Ros...,None,102600,108000,5,5.0,16,backorder,Nhà Sách Tiki/Sách tiếng Việt/Sách văn học/Du ký,1581,Du ký,0,
5,548212,548212,2515413123577,Đà Lạt Một Thời Hương Xa (Du Khảo Văn Hóa Đà L...,https://tiki.vn/product-p548212.html?spid=272396,Đà Lạt Một Thời Hương Xa Đà Lạt được kiến tạo...,None,128000,160000,20,4.8,99,instock,Nhà Sách Tiki/Sách tiếng Việt/Sách văn học/Du ký,2104,Sách tiếng Việt,0,Nguyễn Vĩnh Nguyên
6,113353587,113353587,5776767057441,Nước Đức Từ A Đến Z,https://tiki.vn/product-p113353587.html?spid=1...,“Nước Đức từ A đến Z” là một cuốn sách mới của...,None,94100,119000,21,4.8,55,instock,Nhà Sách Tiki/Sách tiếng Việt/Sách văn học/Du ký,504,Du ký,0,Lê Quang
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15258,576126,576126,2516655198255,Căn Phòng Khóa,https://tiki.vn/product-p576126.html?spid=2459...,Căn Phòng Khóa “Căn Phòng Khóa” là một cuốn ti...,None,131000,146000,10,0.0,0,backorder,Nhà Sách Tiki/Sách tiếng Việt/Sách văn học/Tru...,2104,Root,0,Emma Donoghue
15260,199977123,199977123,7849143177077,Chẳng Cũng Khoái Ru,https://tiki.vn/product-p199977123.html?spid=1...,Chẳng Cũng Khoái RuHầu như chúng ta chẳng bao ...,None,122000,122000,0,0.0,0,backorder,Nhà Sách Tiki/Sách tiếng Việt/Sách văn học/Tru...,54,Truyện ngắn - Tản văn - Tạp Văn,0,B.S Đỗ Hồng Ngọc
15261,185005867,185005867,9986526855908,Tác Phẩm Chọn Lọc - Văn Học Mỹ - Moby Dick Cá ...,https://tiki.vn/product-p185005867.html?spid=1...,Tác Phẩm Chọn Lọc - Văn Học Mỹ - Moby Dick...,None,288000,320000,10,5.0,1,backorder,Nhà Sách Tiki/Sách tiếng Việt/Sách văn học/Tru...,179,Root,0,Herman Melville
15262,2739217,2739217,6625626588233,Cung Đường Mê,https://tiki.vn/product-p2739217.html?spid=204...,"Cung Đường MêCung đường mê gồm 14 truyện ngắn,...",None,38400,48000,20,0.0,0,backorder,Nhà Sách Tiki/Sách tiếng Việt/Sách văn học/Tru...,1622,Sách tiếng Việt,0,Đặng Lưu San


#### **Test tốc độ crawl**

In [311]:
book_test = []

def get_book_info(raw_book, book):
    book_info = {
                'id': raw_book['id'], 
                'master_id': raw_book['master_id'],
                'sku': raw_book['sku'],
                'name': raw_book['name'],
                'short_url': raw_book['short_url'],
                'short_description': raw_book['short_description'],
                'book_cover': raw_book['book_cover'],
                'price': raw_book['price'],
                'original_price': raw_book['original_price'],
                'discount_rate': raw_book['discount_rate'],
                'rating_average': raw_book['rating_average'],
                'review_count': raw_book['review_count'],
                'inventory_type': raw_book['inventory_type'],
                'productset_group_name': raw_book['productset_group_name'],
                'day_ago_created': raw_book['day_ago_created'],
                'categories': raw_book['categories']['name'] if 'categories' in book else [],
                'all_time_quantity_sold': raw_book['all_time_quantity_sold'] if 'all_time_quantity_sold' in book else 0

                }
    name_authors = []
    if 'authors' in raw_book:
        for author in raw_book['authors']:
            name_authors.append(author['name'])
    book_info['authors'] = ', '.join(name_authors)

    if 'specifications' in book:
        for i in book['specifications'][0]['attributes']:
            book_info[i['code']] = i['code']
            book_info[i['value']] = i['value']
    return book_info

for cate in books_id_all_cate[:1]:
    for book in cate['Book_id']:
        url = f'https://tiki.vn/api/v2/products/{book["id"]}'
        try:
            response = httpx.get(url)
            print(response.status_code)
            if response.status_code == 200:
                raw_book = response.json()
                book_test.append(get_book_info(raw_book, book))              
        except:
            # book_test.append(raw_book)
            print(f'ERROR {response.status_code}:', book)
    

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


**Có vẻ như ta đã lấy được hết các dòng**

In [312]:
len(book_test)

195

In [313]:
count = 0
book_test_copy = book_test.copy()
for i in book_test:
    if i!= None:
        count+=1
    else: book_test_copy.remove(i)

len(book_test_copy)

195

**Có khoảng 195 id và ta lấy được hết**

In [274]:
assert len(book_test) == len(books_id_all_cate[0]['Book_id'])

**Thử lại với multithread**

In [309]:
import concurrent.futures 
import time

NUMBER_OF_WORKERS = 4
        
def handle_get_book_info(book):
    url = f'https://tiki.vn/api/v2/products/{book["id"]}'
    try:    
        response = httpx.get(url)   
        print(response.status_code)
        if response.status_code == 200: #and 'content-type' in response.headers 
            #and 'application/json' in response.headers['content-type']):
            raw_book = response.json()
            
            book_info = get_book_info(raw_book, book)
            return book_info
        # response.raise_for_status()
        
    except Exception as e:
        # df.append(book)
        # book_info = response.json()
        # if response.status_code == 429:
        #     time.sleep(int(response.headers["Retry-After"]))
        print(f'ERROR {str(e)}: ', book['id'])
        # elif response.status_code == 200:
        #     book_info = get_book_info(response.json(), book)
        # book_info = raw_book
        # return book_info
        # return response.json()
    # return book_info
        
test_df = []

with concurrent.futures.ThreadPoolExecutor(max_workers=NUMBER_OF_WORKERS) as executor:
    all_params = list(book for cate in books_id_all_cate[:1] 
                      for book in cate['Book_id'] )
    for future in concurrent.futures.as_completed(
        executor.submit(handle_get_book_info, params)
        for params in all_params
    ):
        # result = future.result()
        test_df.append(future.result())
        #print(result)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200200

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
ERROR Expecting value: line 1 column 1 (char 0):  117029310
200
200
ERROR Expecting value: line 1 column 1 (char 0):  192295472
200
ERROR Expecting value: line 1 column 1 (char 0):  127033451
200
200
ERROR Expecting value: line 1 column 1 (char 0):  195832987
200
ERROR Expecting value: line 1 column 1 (char 0):  105278809
200
ERROR Expecting value: line 1 column 1 (char 0):  113583666
200
ERROR Expecting value: line 1 column 1 (char 0):  195304647
200
ERROR Expecting value: line 1 column 1 (char 0):  8234987
200
ERROR 

**Với 10 threads ta lấy được khoảng 130 dòng, tốc độ tăng lên đáng kể: 1m25s -> 25s**
<br>**Tiếp tục thử lại với 5 threads**

|Threads|Số dòng lấy được|Tốc độ|
| -- | -- | -- |
|10|130|25s|
|5|140|38s|
|2|195|59s|

429 error -> too many requests

In [310]:
# len(test_df)
count = 0
test = test_df.copy()
for i in test_df:
    if i!= None:
        # print(i['id'])
        count+=1
    else: 
        test.remove(i)
len(test)

120

**Vậy ta sẽ thử trên toàn bộ dữ liệu với 2 threads**

In [290]:
test_df_second = []

with concurrent.futures.ThreadPoolExecutor(max_workers=NUMBER_OF_WORKERS) as executor:
    all_params = list(book for cate in books_id_all_cate[:20] 
                      for book in cate['Book_id'] )
    for future in concurrent.futures.as_completed(
        executor.submit(handle_get_book_info, params)
        for params in all_params
    ):
        test_df_second.append(future.result())

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
ERROR 200:  192295472
200
ERROR 200:  117029310
200
ERROR 200:  127033451
200
ERROR 200:  195832987
200
ERROR 200:  165038801
200
ERROR 200:  113583666
200
ERROR 200:  105278809
200
ERROR 200:  195304647
200
ERROR 200:  8234987
200
ERROR 200:  190230191
200
ERROR 200:  77221042
200
ERROR 200:  178033006
200
ERROR 200:  196514680
200
ERROR 200:  179196335
200
ERROR 200:  54144906
200
ERROR 200:  109862021
200
ERROR 200:  361817
200
ERROR 200:  34755580
200
ERROR 200:  33402238
200
ERROR 200:  173266960
200
ERROR

UnboundLocalError: local variable 'response' referenced before assignment

In [299]:
count = 0
test = test_df_second.copy()
for i in test_df_second:
    if i!= None:
        # print(i['id'])
        count+=1
    else: 
        test.remove(i)
len(test)

5667